In [1]:
import sys

sys.path.append('..')

In [2]:
from VisionSystem import *
from VisionSystem.DetectionModel import *

def load_or_create_new_threshblob(path):
    try:
        model = ThreshBlob.load(path)
        print("Loaded " + path)
    except Exception:
        model = ThreshBlob()
    finally:
        return model

model_names = ["ball", "obstacle", "blue_goal", "yellow_goal"]
detection_models = {
    model_name: load_or_create_new_threshblob("../models/" + model_name + ".threshblob.pkl") \
                    for model_name in model_names
}

objects_to_track = {
    name: VisualObject(real_size=(0.043, 0.043, 0.043), detection_model=model, result_limit=1)
            for name, model in detection_models.items()
}

stream = VideoStream(downsample_scale=5, crop=((0.13, 0), (.9, 1)))

Loaded ../models/ball.threshblob.pkl
Loaded ../models/obstacle.threshblob.pkl
Loaded ../models/blue_goal.threshblob.pkl
Loaded ../models/yellow_goal.threshblob.pkl


In [3]:
# import cv2
# bgr = cv2.imread('../data/calibration/2.png')
# crop = (0.13, 0), (.9, 1)
# height, width, _ = bgr.shape
# bgr = bgr[:, int(0.13 * width):int(.9 * width)]
vision_system = VisionSystem(objects_to_track, resolution=stream.resolution)

def setup_vision_system(resolution):
    objects_to_size_and_result_limit = [
        ("ball", (0.043, 0.043, 0.043), 1),
        ("obstacle", (0.18, 0.18, 0.2), None),
        # 30 centimetres long, 10 cm high? i guess
        ("blue_goal", (0.3, 0.3, 0.1), 1),
        ("yellow_goal", (0.3, 0.3, 0.1), 1)
    ]

    return VisionSystem(
        resolution=resolution,
        objects_to_track={
            name: VisualObject(
                real_size=size,
                detection_model=ThreshBlob.load(
                    relpath("models", f"{name}.threshblob.pkl")),
                result_limit=result_limit
            ) for name, size, result_limit in objects_to_size_and_result_limit
        }
    )

In [4]:
from DisplayPane import DisplayPane
from DisplayPane.Interactor.VisionSystemTuner import VisionSystemTuner

display = DisplayPane(
    video_stream=stream,
    vision_system=vision_system,
    interactors=[VisionSystemTuner(vision_system)]
)
display

../VisionSystem/DetectionModel/ThreshBlob/ThreshBlob.py:22: RuntimeWarning: overflow encountered in exp
  return (0.1407675 * np.exp(0.85425173 * img_dist / RESCALE / frac_of_full_res) + 0.79365708) * DIST_PER_POINT


DisplayPane(children=(HBox(children=(VBox(children=(Figure(fig_margin={'top': 60, 'bottom': 60, 'left': 60, 'r…

../VisionSystem/DetectionModel/DetectionResult.py:29: RuntimeWarning: overflow encountered in long_scalars
  return 0.5*np.abs(np.dot(x, np.roll(y, 1))-np.dot(y, np.roll(x, 1)))


In [5]:
# RUN THIS CELL TO SAVE THE MODELs tinkered with
for model_name in model_names:
    detection_models[model_name].save("../models/" + model_name + ".threshblob.pkl")

In [6]:
import cv2

bgr = next(iter(stream)).get(ColorSpaces.BGR)

cv2.imwrite('img.png', bgr)

True

In [7]:
bgr.shape

(256, 271, 3)